In [1]:
from src.data import GetData
daily_returns = GetData.daily_returns(20240101,20240602)

In [5]:
from src.data import DataBlock
from typing import Literal
from copy import deepcopy
import torch
import torch.nn.functional as F
def nday_fut_ret(daily_returns : DataBlock , nday : int = 10 , lag : Literal[1,2] = 2 , step : int = 5 , retain_shape = False ,
                 auto_rename = True):
    assert lag > 0 , f'lag must be positive : {lag}'
    block = deepcopy(daily_returns).as_tensor()
    block.values = F.pad(block.values[:,lag:] , (0,0,0,0,0,lag),value=torch.nan)

    new_value = block.values.unfold(1 , nday , step).exp().prod(dim = -1) - 1
    new_date  = block.date[::step][:new_value.shape[1]]
    secid     = block.secid
    feature   = block.feature

    if auto_rename: feature = [f'{nday}day_{feat}_ret_lag{lag-1}' for feat in feature]
    new_block = DataBlock(new_value , secid , new_date , feature)
    if retain_shape: new_block.align_date(block.date)
    return new_block

In [6]:
block = nday_fut_ret(daily_returns)

In [7]:
block.to_dataframe()

10day_close_ret_lag1
secid  date                          
1      20240102              0.003814
       20240109              0.026858
       20240116              0.025671
       20240123              0.031273
       20240130              0.158645
...                               ...
689009 20240412              0.284709
       20240419              0.287006
       20240426              0.093814
       20240508              0.039070
       20240515             -0.063290

[92232 rows x 1 columns]

: 

In [45]:
import numpy as np
np.arange(6).reshape(3,2)[0]

array([0, 1])

In [42]:
np.arange(10).tile(2)

AttributeError: 'numpy.ndarray' object has no attribute 'tile'

In [38]:
block.secid.repeat(2) , block.date.repeat(1,by=2)

TypeError: 'by' is an invalid keyword argument for repeat()

In [22]:
new_block.values[:10,:,0,0]

tensor([[ 0.0146,     nan,     nan,     nan,     nan, -0.0019,     nan,     nan,
             nan,     nan,  0.0179,     nan,     nan,     nan,     nan,  0.0582,
             nan,     nan,     nan,     nan,  0.1431,     nan,     nan,     nan,
             nan,  0.0895,     nan,     nan,     nan,     nan, -0.0428,     nan,
             nan,     nan,     nan, -0.0146,     nan,     nan,     nan,     nan,
          0.0128,     nan,     nan,     nan,     nan,  0.0205,     nan,     nan,
             nan,     nan,  0.0014,     nan,     nan,     nan,     nan, -0.0443,
             nan,     nan,     nan,     nan,  0.0233,     nan,     nan,     nan,
             nan,  0.0549,     nan,     nan,     nan,     nan,  0.0043,     nan,
             nan,     nan,     nan,  0.0222,     nan,     nan,     nan,     nan,
          0.0785,     nan,     nan,     nan,     nan,  0.0410,     nan,     nan,
             nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan,
             nan,     nan],


In [ ]:
import pandas as pd
import numpy as np
from .stats_utils.ic_perf import calc_ic
from .stats_utils.grouping import calc_group_perf


def calc_decay_pnl(factor_val_df, ret_type, yend_ed, bm_index_nm, ret_range_type, price_type, ic_type, lag_num):
    decay_pnl_df = []
    for lag in range(lag_num + 1):
        decay_pnl = calc_ic(factor_val_df, ret_type, yend_ed, bm_index_nm, ret_range_type, price_type, ic_type, lag)
        decay_pnl = decay_pnl.stack().rename("ic").reset_index(drop=False)
        decay_pnl["lag_type"] = "lag{0}".format(lag)
        decay_pnl_df.append(decay_pnl)
    decay_pnl_df = pd.concat(decay_pnl_df, axis=0)
    #
    ic_mean = decay_pnl_df.groupby(["factor_name", "lag_type"])["ic"].mean().rename("ic_mean").reset_index(drop=False)
    return ic_mean


def calc_decay_grp_perf(factor_val_df, ret_type, freq_type, yend_ed, grp_bm_index, group_nums, ret_range_type,
                        price_type, lag_num):
    decay_grp_perf = []
    for lag in range(lag_num + 1):
        grp_perf = calc_group_perf(factor_val_df, ret_type, yend_ed, grp_bm_index, group_nums, ret_range_type,
                                   price_type, lag)
        grp_perf["lag_type"] = "lag{0}".format(lag)
        decay_grp_perf.append(grp_perf)
    decay_grp_perf = pd.concat(decay_grp_perf, axis=0)
    #
    freq_days_dict = {"day": 245, "week": 50, "month": 12}
    assert freq_type in freq_days_dict.keys()
    decay_grp_ret_mean = decay_grp_perf.groupby(["factor_name", "group", "lag_type"])["group_ret"].mean() * freq_days_dict[freq_type]
    deacy_grp_rert_std = decay_grp_perf.groupby(["factor_name", "group", "lag_type"])["group_ret"].std() * \
                       np.sqrt(freq_days_dict[freq_type])
    decay_grp_ret_ir = decay_grp_ret_mean / deacy_grp_rert_std
    rtn = pd.concat(
        (
            decay_grp_ret_mean.rename("decay_grp_ret"),
            decay_grp_ret_ir.rename("decay_grp_ir")
         ),
        axis=1, sort=True).rename_axis("stats_name", axis="columns").stack().rename("stats_value").reset_index(drop=False)
    return rtn


In [1]:
from src import API

API.DataAPI.update()
API.DataAPI.prepare_train_data()
API.Predictor.update_factors()

Update Files
Tue Jun  4 23:42:09 2024 : Updated ~ data\DataBase\DB_information\calendar.feather Done! Cost 0.03 Secs
Tue Jun  4 23:42:09 2024 : Updated ~ data\DataBase\DB_information\description.feather Done! Cost 0.04 Secs
Tue Jun  4 23:42:09 2024 : Updated ~ data\DataBase\DB_information\st.feather Done! Cost 0.02 Secs
Tue Jun  4 23:42:12 2024 : Updated ~ data\DataBase\DB_information\industry.feather Done! Cost 2.40 Secs
Tue Jun  4 23:42:16 2024 : Updated ~ data\DataBase\DB_information\concepts.feather Done! Cost 4.45 Secs
Tue Jun  4 23:42:17 2024 : Updated ~ data\DataBase\DB_models\longcl_exp\2024\longcl_exp.20240603.feather Done! Cost 0.09 Secs
Tue Jun  4 23:42:17 2024 : Updated ~ data\DataBase\DB_models\risk_exp\2024\risk_exp.20240604.feather Done! Cost 0.05 Secs
Tue Jun  4 23:42:19 2024 : Updated ~ data\DataBase\DB_trade\day\2024\day.20240604.feather Done! Cost 0.05 Secs
Tue Jun  4 23:42:19 2024 : Updated ~ data\DataBase\DB_trade\5day\2024\5day.20240604.feather Done! Cost 0.24 Sec

In [2]:
from src import API
API.Predictor.update_factors()

Load  2 DataBlocks...... finished! Cost 0.11 secs
Align 2 DataBlocks...... finished! Cost 0.17 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
20231201 new
Load  2 DataBlocks...... finished! Cost 0.03 secs
Align 2 DataBlocks...... finished! Cost 0.15 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
20231201 new
Load  2 DataBlocks...... finished! Cost 0.03 secs
Align 2 DataBlocks...... finished! Cost 0.17 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
20231201 new


In [2]:
from src import API
API.ModelTestor.new('ts_mixer').try_metrics()

Load  2 DataBlocks ...

... cost 0.22 secs
Align 2 DataBlocks ...... cost 0.21 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
x shape is torch.Size([5064, 30, 6])
y shape is torch.Size([5064, 1])
Test Forward Success
metrics :  Metrics.MetricOutput(loss=tensor(1.0688, grad_fn=<AddBackward0>), score=-0.015420470386743546, loss_item=1.068833589553833, penalty=0.0, losses=tensor(1.0688, grad_fn=<ExpBackward0>))
Test Metrics Success
